In [ ]:
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

from utils import GLOBAL, functions

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [16]:
data_df = pd.DataFrame()

yearStart = 2014
yearEnd = 2021

WEEKLY_BASE_URL = "https://raw.githubusercontent.com/fantasydatapros/data/master/weekly/{year}/week{week}.csv"

for year in range(yearStart, yearEnd):
    for week in range(1, 18):
        weekly_df = pd.read_csv(WEEKLY_BASE_URL.format(year=year, week=week))
        weekly_df['Year'] = year
        weekly_df['Week'] = week
        weekly_df['Usage'] = (weekly_df['PassingAtt'] + weekly_df['RushingAtt'] + weekly_df['Tgt'])
        data_df = pd.concat([data_df, weekly_df])

In [21]:
data_df = data_df.groupby(['Player', 'Pos', 'Tm', 'Year'], as_index=False)\
    .agg({
        'Usage': np.sum,
        'PassingYds': np.sum,
        'PassingTD': np.sum,
        'PassingAtt': np.sum,
        'RushingAtt': np.sum,
        'RushingYds': np.sum,
        'RushingTD': np.sum,
        'Rec': np.sum,
        'Tgt': np.sum,
        'ReceivingYds': np.sum,
        'ReceivingTD': np.sum,
        'PPRFantasyPoints': np.sum,
        'StandardFantasyPoints': np.sum,
        'HalfPPRFantasyPoints': np.sum
    })